In [1]:
from qutip import *
import matplotlib.pyplot as plt
import numpy as np
from helper_code_qutip import *
from scipy import integrate
import scipy.io
import time

In [2]:
N = 2
d = 2**N
dims = [2]*N

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)
eigenergies,eigstates=H_S.eigenstates()
number = len(eigenergies)

mu2 = 0
gamma1 = 1
tb = 0.01

In [3]:
create_sm_list = [create_sm(N,i + 1) for i in range(N)]
beta = 1
rho_th = (-beta*H_S).expm()/((-beta*H_S).expm()).tr() #left thermal density matrix
limit_value = 700
b = 500

integral11=np.empty((number,number),dtype=np.cdouble) #stores J * N integral for left bath
integral12=np.empty((number,number),dtype=np.cdouble) # stores J integral (just to check) for the left bath
integral21=np.empty((number,number),dtype=np.cdouble) #stores J*N integral for right bath
integral22=np.empty((number,number),dtype=np.cdouble)

        #print("Integral calculations at beta2 = {} and e = {} are : ".format(beta2,e))

for i in range(number):
    for k in range(number):
                freq=eigenergies[k]-eigenergies[i]
                #print(f"Absolute frequency  for i = {i}, k = {k} is ",np.absolute(freq))
                #print(i,k,freq)
                if( np.absolute(freq) >= 1/10**10):
                    integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu2,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                    integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #left bath done
                    """integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                    integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #right bath"""
        
                if (np.absolute(freq)<=1/10**10):  #The problem is arising here....
                    integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu2,gamma1),limit=limit_value)[0]
                    integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma1),limit=limit_value)[0]
                    """integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value)[0]
                    integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma2),limit=limit_value)[0]"""
                
            
            #expected=1.0j*(eigenergies[k]-eigenergies[i])/(2*tb*tb)
        #        print(i,k,integral2[i,k],expected)
    
    
        # PAY ATTENTION TO THE WAY THESE COEFFICIENTS ARE BEING COMPUTED
    
constant12=np.empty((number,number),dtype=np.cdouble)
constant11=np.empty((number,number),dtype=np.cdouble)
"""constant21=np.empty((number,number),dtype=np.cdouble)
constant22=np.empty((number,number),dtype=np.cdouble)"""
        
        
        
for i in range(number):
        for k in range(number):
                constant12[i,k]=integral12[i,k]+integral11[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma1)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu2,gamma1))    #full coefficient created this is nbar+1
                constant11[i,k]=integral11[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta,mu2,gamma1)                                       # the full coefficient is created
                
                """constant22[i,k]=integral22[i,k]+integral21[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma2)+func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2))    #full coefficient created this is nbar+1
                constant21[i,k]=integral21[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2)   # the full coefficient is created"""

c:\Users\DELL\OneDrive\Desktop\Term paper and Proj report\ICTS Project\SearchingLindbladians\python\helper_code_qutip.py:205: RuntimeWarning: divide by zero encountered in scalar divide
  return 1/(np.exp(beta*(omega-mu))-1)
C:\Users\DELL\AppData\Local\Temp\ipykernel_38980\3859668315.py:26: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu2,gamma1),limit=limit_value)[0]


In [4]:
epsilon = 0.01

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(N):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list[l]*proj_y,create_sm_list[l].dag())*constant11[k,y]
                sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list[l].dag(),proj_y*create_sm_list[l]*proj_k)*constant12[y,k]
                sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum

In [5]:
b = np.zeros((number),dtype=complex)
q,r = np.linalg.qr(A.T)

#print(r)
diag_r = [r[i][i] for i in range(number)]
print(diag_r)

[(-0.00024983125307470204+0j), (-0.00021881871679398177+0j), (-0.0002184643234733033+0j), (-2.6904790350525558e-20+0j)]


In [6]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]

In [7]:
A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])
b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)
print(np.dot(A_new,x))
print(np.dot(A[-1],x))
x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

#set U matrix whose columns are the eigenvectors of the Hamiltonian

U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

print([x.real for x in  rho_th_diag])
print(x_real)

[6.77626358e-21+0.j 0.00000000e+00+0.j 3.38813179e-21+0.j
 1.00000000e+00+0.j]
(-2.371692252312041e-20+0j)
[0.5351178428087139, 0.1968588529733373, 0.1956029793964662, 0.07242032482148247]
[0.535117842808714, 0.19685885297333738, 0.19560297939646623, 0.07242032482148242]


In [12]:
rho_comp2 = np.dot(U,np.dot(rho,U.T.conjugate()))
def L2_red(rho,eigstates):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(N):
                op = commutator(rho*proj_i*create_sm_list[l]*proj_k,create_sm_list[l].dag())*constant11[i,k] + commutator(create_sm_list[l].dag(),proj_i*create_sm_list[l]*proj_k*rho)*constant12[i,k]
                sum += op
                sum += op.dag()

            """for l in range(NL2):
                op = commutator(rho*proj_i*create_sm_list_right[l]*proj_k,create_sm_list_right[l].dag())*constant21[i,k] + commutator(create_sm_list_right[l].dag(),proj_i*create_sm_list_right[l]*proj_k*rho)*constant22[i,k]
                sum += op
                sum += op.dag()"""
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

In [13]:
L2_redfield = L2_red(rho_comp2,eigstates)
rho_th_qutip = Qobj(rho_th)
rho_th_qutip.dims = [dims,dims]

rho_comp_qutip = Qobj(rho_comp2)
rho_comp_qutip.dims = [dims,dims]
print(tracedist(rho_th_qutip,rho_comp_qutip))

rho_th_check = np.zeros((number,number),dtype=complex)
for i in range(number):
    for k in range(number):
        rho_th_check[i,k] = rho_th[i,k]

#print(rho_th_check)
#print(rho_th_new)
#print(L2_redfield)
#print(rho_th_check)

data_dict = {"dm_ness":rho_comp2, "dm_th":rho_th_new,"L2_red":L2_redfield, "dm_th_check":rho_th_check}

scipy.io.savemat(f'../matlab/coh_data_N = {N}_1.mat',data_dict)

1.3881217231918663e-16
